<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Mortgage Default Machine Learning Model</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
</table>

This notebook walks you through these steps:
- Access the data
- Cleanse data for analysis
- Explore data
- Build a classification model
- Save the model in the ML repository with associated meta data


### Step 1: Load Data as Spark Dataframe

Read the three Mortgage files - Insert them as SPARK Dataframes

In [1]:
from pyspark.sql import SQLContext

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190221212537-0001
KERNEL_ID = 9664c122-c893-41b3-8cf5-39f97948e165


In [2]:
import ibmos2spark
# @hidden_cell
credentials = 

configuration_name = 'os_b8c6e2907b17487a99b6a1834b9d7e35_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

# Add asset from file system
df_data_1 = SQLContext(sc).read.csv(cos.url('Mortgage_Customer.csv', 'democrditoshipotecarios-donotdelete-pr-ds3e6cthycjqmq'),
                                    header='true', inferSchema = 'true')
df_data_1.take(5)

[Row(ID=100522, Income=43982, AppliedOnline='YES', Residence='Owner Occupier', Yrs_at_Current_Address=13, Yrs_with_Current_Employer=11, Number_of_Cards=2, Creditcard_Debt=1055, Loans=0, Loan_Amount=9405),
 Row(ID=101756, Income=59944, AppliedOnline='YES', Residence='Owner Occupier', Yrs_at_Current_Address=20, Yrs_with_Current_Employer=11, Number_of_Cards=2, Creditcard_Debt=3894, Loans=0, Loan_Amount=9880),
 Row(ID=101354, Income=57718, AppliedOnline='YES', Residence='Owner Occupier', Yrs_at_Current_Address=25, Yrs_with_Current_Employer=16, Number_of_Cards=2, Creditcard_Debt=1555, Loans=1, Loan_Amount=6285),
 Row(ID=100512, Income=45621, AppliedOnline='YES', Residence='Owner Occupier', Yrs_at_Current_Address=1, Yrs_with_Current_Employer=19, Number_of_Cards=1, Creditcard_Debt=1878, Loans=0, Loan_Amount=9260),
 Row(ID=100537, Income=45081, AppliedOnline='NO', Residence='Owner Occupier', Yrs_at_Current_Address=14, Yrs_with_Current_Employer=15, Number_of_Cards=2, Creditcard_Debt=713, Loans=

In [3]:
# Add asset from file system
df_data_2 = SQLContext(sc).read.csv(cos.url('Mortgage_Property.csv', 'democrditoshipotecarios-donotdelete-pr-ds3e6cthycjqmq'),
                                    header='true', inferSchema = 'true')
df_data_2.take(5)

[Row(ID=100529, SalePrice=150000, Location=110),
 Row(ID=100458, SalePrice=475000, Location=110),
 Row(ID=101388, SalePrice=260000, Location=120),
 Row(ID=101472, SalePrice=340000, Location=100),
 Row(ID=100722, SalePrice=242500, Location=110)]

In [4]:
# Add asset from file system
df_data_3 = SQLContext(sc).read.csv(cos.url('Mortgage_Default.csv', 'democrditoshipotecarios-donotdelete-pr-ds3e6cthycjqmq'), 
                                    header='true', inferSchema = 'true')
df_data_3.take(5)

[Row(ID=101600, MortgageDefault='YES'),
 Row(ID=101731, MortgageDefault='NO'),
 Row(ID=100548, MortgageDefault='YES'),
 Row(ID=101472, MortgageDefault='NO'),
 Row(ID=100562, MortgageDefault='YES')]

In [5]:
customer = df_data_1
proper = df_data_2
default = df_data_3

In [6]:
customer.cache()
proper.cache()
default.cache()

DataFrame[ID: int, MortgageDefault: string]

In [7]:
print("Customer dataframe contains these fields:")
print(customer.schema.names)
print("")
print("Property dataframe contains these fields:")
print(proper.schema.names)
print("")
print("Default dataframe contains these fields:")
print(default.schema.names)

Customer dataframe contains these fields:
['ID', 'Income', 'AppliedOnline', 'Residence', 'Yrs_at_Current_Address', 'Yrs_with_Current_Employer', 'Number_of_Cards', 'Creditcard_Debt', 'Loans', 'Loan_Amount']

Property dataframe contains these fields:
['ID', 'SalePrice', 'Location']

Default dataframe contains these fields:
['ID', 'MortgageDefault']


View the data within the three data frames created above. 

In [8]:
customer.show(5)
proper.show(5)
default.show(5)

+------+------+-------------+--------------+----------------------+-------------------------+---------------+---------------+-----+-----------+
|    ID|Income|AppliedOnline|     Residence|Yrs_at_Current_Address|Yrs_with_Current_Employer|Number_of_Cards|Creditcard_Debt|Loans|Loan_Amount|
+------+------+-------------+--------------+----------------------+-------------------------+---------------+---------------+-----+-----------+
|100522| 43982|          YES|Owner Occupier|                    13|                       11|              2|           1055|    0|       9405|
|101756| 59944|          YES|Owner Occupier|                    20|                       11|              2|           3894|    0|       9880|
|101354| 57718|          YES|Owner Occupier|                    25|                       16|              2|           1555|    1|       6285|
|100512| 45621|          YES|Owner Occupier|                     1|                       19|              1|           1878|    0|     

### Step 2: Merge Files

In [9]:
# Join the Customer, Property and Default tables together with ID being the key field.
# Keep all fields from the Customer and Default tables but only SalePrice and Location from the Property table

merged = customer.join(proper, customer['ID'] == proper['ID'])\
                   .join(default, customer['ID']==default['ID'])\
                   .select(customer['ID'],customer['Income'],customer['AppliedOnline'],customer['Residence'],\
                           customer['Yrs_at_Current_Address'],customer['Yrs_with_Current_Employer'],customer['Number_of_Cards'],\
                           customer['Creditcard_Debt'],customer['Loans'],customer['Loan_Amount'],\
                           proper['SalePrice'], proper['Location'], default['MortgageDefault'])

# Preview  5 rows
merged.toPandas().head()

,ID,Income,AppliedOnline,Residence,Yrs_at_Current_Address,Yrs_with_Current_Employer,Number_of_Cards,Creditcard_Debt,Loans,Loan_Amount,SalePrice,Location,MortgageDefault
0,100522,43982,YES,Owner Occupier,13,11,2,1055,0,9405,500000,110,NO
1,101756,59944,YES,Owner Occupier,20,11,2,3894,0,9880,750000,110,NO
2,101354,57718,YES,Owner Occupier,25,16,2,1555,1,6285,155000,130,YES
3,100512,45621,YES,Owner Occupier,1,19,1,1878,0,9260,195000,100,YES
4,100537,45081,NO,Owner Occupier,14,15,2,713,1,8430,140000,110,NO


In [10]:
merged = merged.drop('ID')

In [11]:
merged.toPandas().head()

,Income,AppliedOnline,Residence,Yrs_at_Current_Address,Yrs_with_Current_Employer,Number_of_Cards,Creditcard_Debt,Loans,Loan_Amount,SalePrice,Location,MortgageDefault
0,43982,YES,Owner Occupier,13,11,2,1055,0,9405,500000,110,NO
1,59944,YES,Owner Occupier,20,11,2,3894,0,9880,750000,110,NO
2,57718,YES,Owner Occupier,25,16,2,1555,1,6285,155000,130,YES
3,45621,YES,Owner Occupier,1,19,1,1878,0,9260,195000,100,YES
4,45081,NO,Owner Occupier,14,15,2,713,1,8430,140000,110,NO


### Step 3: Simple Data Preparation - Rename some columns and ensure correct data types 
This step is to remove spaces from columns names

In [12]:
merged = merged.withColumnRenamed("Yrs_at_Current_Address", "YearCurrentAddress").withColumnRenamed("Yrs_with_Current_Employer","YearsCurrentEmployer")\
                .withColumnRenamed("Number_of_Cards","NumberOfCards").withColumnRenamed("Creditcard_Debt","CCDebt").withColumnRenamed("Loan_Amount", "LoanAmount")
merged.toPandas().head(3)

,Income,AppliedOnline,Residence,YearCurrentAddress,YearsCurrentEmployer,NumberOfCards,CCDebt,Loans,LoanAmount,SalePrice,Location,MortgageDefault
0,43982,YES,Owner Occupier,13,11,2,1055,0,9405,500000,110,NO
1,59944,YES,Owner Occupier,20,11,2,3894,0,9880,750000,110,NO
2,57718,YES,Owner Occupier,25,16,2,1555,1,6285,155000,130,YES


Check data types and re-cast numeric fields to **Integers**

In [13]:
merged.dtypes

[('Income', 'int'),
 ('AppliedOnline', 'string'),
 ('Residence', 'string'),
 ('YearCurrentAddress', 'int'),
 ('YearsCurrentEmployer', 'int'),
 ('NumberOfCards', 'int'),
 ('CCDebt', 'int'),
 ('Loans', 'int'),
 ('LoanAmount', 'int'),
 ('SalePrice', 'int'),
 ('Location', 'int'),
 ('MortgageDefault', 'string')]

In [14]:
merged = merged.select(merged.Income.cast('integer'),merged.AppliedOnline,merged.Residence,\
                   merged.YearCurrentAddress.cast('integer'),merged.YearsCurrentEmployer.cast('integer'),\
                   merged.NumberOfCards.cast('integer'),merged.CCDebt.cast('integer'),merged.Loans.cast('integer'),\
                   merged.LoanAmount.cast('integer'),merged.SalePrice,merged.Location,merged.MortgageDefault)
merged.dtypes

[('Income', 'int'),
 ('AppliedOnline', 'string'),
 ('Residence', 'string'),
 ('YearCurrentAddress', 'int'),
 ('YearsCurrentEmployer', 'int'),
 ('NumberOfCards', 'int'),
 ('CCDebt', 'int'),
 ('Loans', 'int'),
 ('LoanAmount', 'int'),
 ('SalePrice', 'int'),
 ('Location', 'int'),
 ('MortgageDefault', 'string')]

### Step 4: Data Exploration

1) Obtain some data shape summaries in terms of number of fields and records <br>
2) Perform some exploratory analysis of distributions, scatterplots using two different graphics packages

#### Feel free to play around with the charts to explore other features that are in the data set

In [15]:
print("There are " + str(merged.count()) + " records and " + str(len(merged.columns)) + " fields in the dataset.")

There are 419 records and 12 fields in the dataset.


In [16]:
import brunel
df = merged.toPandas()
%brunel data('df') bar x(Residence) y(Income) mean(CCDebt) color(MortgageDefault) stack tooltip(Income) | x(YearCurrentAddress) y(YearsCurrentEmployer) point color(MortgageDefault) tooltip(YearCurrentAddress, YearsCurrentEmployer) :: width=1100, height=400 

<IPython.core.display.Javascript object>

In [17]:
from pixiedust.display import *
display(merged)

### Step 5: Build the Spark pipeline and the Random Forest model
"Pipeline" is an API in SparkML that's used for building models.
Additional information on SparkML: https://spark.apache.org/docs/2.0.2/ml-guide.html

In [18]:
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Prepare string variables so that they can be used by the algorithm
stringIndexer1 = StringIndexer(inputCol='AppliedOnline', outputCol='AppliedOnlineEncoded')
stringIndexer2 = StringIndexer(inputCol='Residence',outputCol='ResidenceEncoded')
stringIndexer3 = StringIndexer(inputCol='MortgageDefault', outputCol='label')

# Instanciate the algorithm
rf=RandomForestClassifier(labelCol="label", featuresCol="features")


# Pipelines API requires that input variables are passed in  a vector
assembler = VectorAssembler(inputCols=["Income", "AppliedOnlineEncoded", "ResidenceEncoded", "YearCurrentAddress", "YearsCurrentEmployer", "NumberOfCards", \
                                       "CCDebt", "Loans", "LoanAmount", "SalePrice", "Location"], outputCol="features")

pipeline = Pipeline(stages=[stringIndexer1, stringIndexer2, stringIndexer3, assembler, rf])

In [19]:
type(pipeline)

pyspark.ml.pipeline.Pipeline

In [20]:
# Split data into train and test datasets
train, test = merged.randomSplit([80.0,20.0], seed=6)

In [21]:
# Build model based upon the pipeline defined in the above cell
model = pipeline.fit(train)

### Step 6: Score the test data set

In [22]:
results = model.transform(test)
results.toPandas().head(3)

,Income,AppliedOnline,Residence,YearCurrentAddress,YearsCurrentEmployer,NumberOfCards,CCDebt,Loans,LoanAmount,SalePrice,Location,MortgageDefault,AppliedOnlineEncoded,ResidenceEncoded,label,features,rawPrediction,probability,prediction
0,43185,NO,Living with parents/guardian,0,8,1,20,0,9420,305000,130,NO,1.0,3.0,0.0,"[43185.0, 1.0, 3.0, 0.0, 8.0, 1.0, 20.0, 0.0, ...","[14.5534070204, 5.44659297964]","[0.727670351018, 0.272329648982]",0.0
1,43185,YES,Owner Occupier,1,9,2,1646,0,8125,235000,100,NO,0.0,0.0,0.0,"[43185.0, 0.0, 0.0, 1.0, 9.0, 2.0, 1646.0, 0.0...","[14.5635398887, 5.43646011128]","[0.728176994436, 0.271823005564]",0.0
2,43202,YES,Owner Occupier,17,7,1,1412,1,8925,650000,101,NO,0.0,0.0,0.0,"[43202.0, 0.0, 0.0, 17.0, 7.0, 1.0, 1412.0, 1....","[12.7553917422, 7.24460825781]","[0.63776958711, 0.36223041289]",0.0


### Step 7: Model Evaluation 

In [23]:
print('Precision model1 = {:.2f}.'.format(results.filter(results.label == results.prediction).count() / float(results.count())))

Precision model1 = 0.69.


In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
print('Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results)))

Area under ROC curve = 0.68.


In [25]:
evaluatorRF = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluatorRF.evaluate(results)

print("Accuracy = %g" % accuracy)

Accuracy = 0.683824


### Step 9: Create and deploy the model as an API

In [26]:
#Import required WML Libraries
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps, MetaNames

In [27]:
#Import your WML Credentials from your IBM Cloud service
wml_credentials= 

In [28]:
#Authenticate the repository
ml_repository_client = MLRepositoryClient(wml_credentials['url'])
ml_repository_client.authorize(wml_credentials['username'], wml_credentials['password'])

In [29]:
type(model)

pyspark.ml.pipeline.PipelineModel

In [30]:
# Check if props is mandatory
props = MetaProps({MetaNames.AUTHOR_NAME:"Juan Mora", MetaNames.AUTHOR_EMAIL:"jpmora@co.ibm.com"})
model_artifact = MLRepositoryArtifact(model, name="RF-Spark", training_data=train)

In [31]:
#Create schemas for saving the model
saved_model = ml_repository_client.models.save(model_artifact)
saved_model.meta.available_props()

dict_keys(['runtime', 'creationTime', 'trainingDataRef', 'version', 'evaluationMethod', 'lastUpdated', 'trainingDataSchema', 'evaluationMetrics', 'modelType', 'pipelineVersionHref', 'label', 'authorEmail', 'inputDataSchema', 'modelVersionHref', 'authorName'])

In [32]:
#importing libraries necessary for creating the access token to WML
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(wml_credentials['username'], wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

In [33]:
#creating the scoring endpoint
endpoint_instance = wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id']
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken} 

response_get_instance = requests.get(endpoint_instance, headers=header)

In [34]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')
print(endpoint_published_models)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/b958b4a9-fbee-4a79-86dc-02854a9102b0/published_models


In [35]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get = requests.get(endpoint_published_models, headers=header)

In [36]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

print(endpoint_deployments)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/b958b4a9-fbee-4a79-86dc-02854a9102b0/published_models/43a83b88-660f-442e-bfe0-d48ebb709604/deployments


In [37]:
#Deploy the model
payload_online = {"name": "SparkModel", "description": "Mortgage risk using Random Forest", "type": "online"}
response_online = requests.post(endpoint_deployments, json=payload_online, headers=header)
print(response_online.text)

{
  "metadata": {
    "guid": "2516b798-2ec5-4e0a-a105-c66ebc20359c",
    "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/b958b4a9-fbee-4a79-86dc-02854a9102b0/published_models/43a83b88-660f-442e-bfe0-d48ebb709604/deployments/2516b798-2ec5-4e0a-a105-c66ebc20359c",
    "created_at": "2019-02-21T21:30:45.144Z",
    "modified_at": "2019-02-21T21:30:48.522Z"
  },
  "entity": {
    "runtime_environment": "spark-2.3",
    "name": "SparkModel",
    "scoring_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/b958b4a9-fbee-4a79-86dc-02854a9102b0/published_models/43a83b88-660f-442e-bfe0-d48ebb709604/deployments/2516b798-2ec5-4e0a-a105-c66ebc20359c/online",
    "deployable_asset": {
      "name": "RF-Spark",
      "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/b958b4a9-fbee-4a79-86dc-02854a9102b0/published_models/43a83b88-660f-442e-bfe0-d48ebb709604",
      "guid": "43a83b88-660f-442e-bfe0-d48ebb709604",
      "created_at": "2019-02-21T21:30:45.119Z",
      "type": "m

In [38]:
scoring_url = json.loads(response_online.text).get('entity').get('scoring_url')
print(scoring_url)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/b958b4a9-fbee-4a79-86dc-02854a9102b0/published_models/43a83b88-660f-442e-bfe0-d48ebb709604/deployments/2516b798-2ec5-4e0a-a105-c66ebc20359c/online


In [39]:
merged.toPandas().drop(['MortgageDefault'], axis=1).head(5)

,Income,AppliedOnline,Residence,YearCurrentAddress,YearsCurrentEmployer,NumberOfCards,CCDebt,Loans,LoanAmount,SalePrice,Location
0,43982,YES,Owner Occupier,13,11,2,1055,0,9405,500000,110
1,59944,YES,Owner Occupier,20,11,2,3894,0,9880,750000,110
2,57718,YES,Owner Occupier,25,16,2,1555,1,6285,155000,130
3,45621,YES,Owner Occupier,1,19,1,1878,0,9260,195000,100
4,45081,NO,Owner Occupier,14,15,2,713,1,8430,140000,110


In [40]:
mergededited = merged.toPandas().drop(['MortgageDefault'], axis=1)

In [41]:
import numpy as np
import pandas as pd
income=int(input("Income: "))
appliedonline=input("Appplied on line: ")
residence=input("Residence: ")
ycurrentadd=int(input("Years in current address: "))
ycurrentempl=int(input("Years with current employer: "))
ncards=int(input("Number of cards: "))
ccdebt=int(input("Credit card debt: "))
loans=int(input("Loans: "))
loanammount=int(input("Loans ammount: "))
salesprice=int(input("Sales price: "))
location=int(input("Location: "))



new_observation=np.array([[income, appliedonline, residence, ycurrentadd, ycurrentempl, ncards, ccdebt, loans, loanammount, salesprice, location]],dtype=object)
new_observation=pd.DataFrame(new_observation,columns=mergededited.columns)
#new_observation.replace(dictionaries.encode,inplace=True)


payload_scoring={'fields': list(mergededited.columns), 'values': [list(new_observation.values[0])]}
print(payload_scoring)

Income: 120000
Appplied on line: YES
Residence: Public Housing
Years in current addres: 12
Years with current employer: 10
Number of cards: 2
Credit card debt: 500
Loans: 1
Loans ammount: 13450
Sales price: 123678
Location: 110
{'values': [[120000, 'YES', 'Public Housing', 12, 10, 2, 500, 1, 13450, 123678, 110]], 'fields': ['Income', 'AppliedOnline', 'Residence', 'YearCurrentAddress', 'YearsCurrentEmployer', 'NumberOfCards', 'CCDebt', 'Loans', 'LoanAmount', 'SalePrice', 'Location']}


In [42]:
response_scoring=requests.post(scoring_url,json=payload_scoring, headers=header)
response=json.loads(response_scoring.text)
probabilidad_abandono=response['values'][0][15][1]*100

In [43]:
print("Este cliente tiene una probabilidad de",np.round(probabilidad_abandono,2),"% de default en su crédito hipotecario")

Este cliente tiene una probabilidad de 71.47 % de default en su crédito hipotecario


## Summary:
You are now at the end of this notebook and should have successfully:
- Performed basic data preparation on the loaded data
- Explored the data graphically
- Build a Spark Model in the form of a pipeline
- Evaluated the model for accuracy levels
- Stored the model into the IBM ML environment, making it ready for deployment